<a href="https://colab.research.google.com/github/deepartist/DeepFaceLab-Tools/blob/main/DFL_Upscaling_Tools_deepartist.ai_AT_gmail.com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Collection DeepFaceLab Upscaling Tools by deepartist**

## **Sources:** 
* ## GPEN
[bycloud - youtube channel](https://www.youtube.com/channel/UCfg9ux4m8P0YDITTPptrmLg) and 
[Code]( https://github.com/bycloudai/GPEN-colab)

* ## GFPGAN - Towards Real-World Blind Face Restoration with Generative Facial Prior
[Paper](https://arxiv.org/abs/2101.04061) 
[Project Page](https://xinntao.github.io/projects/gfpgan)
[colab](https://colab.research.google.com/drive/1sVsoBd9AjckIXThgtZhGrHRfFI6UUYOo)

* ## Progressive Semantic-Aware Style Transformation for Blind Face Restoration
 [Code](https://github.com/chaofengc/PSFRGAN)
 [Paper](https://arxiv.org/abs/2009.08709)
 [Colab](https://gist.github.com/woctezuma/20e4dd3d49da09bf339d909f29c5527e)



Based on the upscaling Notebook of [Tendeepfake](mailto:tendeepfake@gmail.com
)
## *There is no need for colab pro to use it.*

[Contact me: deepartist](mailto:deepartist.ai@gmail.com)



---



# Install

In [ ]:
#@title Check if you have loaded GPU
gpu = !nvidia-smi -L
gpu= gpu[0]
gpu_mem = !nvidia-smi --query-gpu=memory.total --format=csv
gpu_mem = gpu_mem[1]
gpu_mem_free = !nvidia-smi --query-gpu=memory.free --format=csv
gpu_mem_free = gpu_mem_free[1]
cores = !grep -c ^processor /proc/cpuinfo
cores = cores[0]
mhz = !grep "^[c]pu MHz" /proc/cpuinfo
mhz = mhz[0].split(":")[-1]
mhz
!echo "$gpu"
!echo Total Memory: "$gpu_mem" Free: "$gpu_mem_free"
!echo CPU: "$cores" cores @"$mhz" MHz

**Mount your Google drive with auth key**

1. Connect to your google drive to be able to import and export files to/from google drive
2. Click on Link below
3. Login and confirm
4. Ctrl+V auth key and hit return key

**OR mount your Google drive permantly**

1. Create new notebook
2. Change runtime to GPU
3. Click on files, and mount your google drive permantly 
4. Click in first cell
5. Select all cells with shift+down
6. Copy all cells from this notebook
7. No more auth token coyping

**OR SIMPLY SKIP** if you dont want import and export files to/from google drive

In [ ]:
#@title Connect to your gdrive IF you not have permantly mount it

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Clone DFL Git and install needed dependencies
!git clone https://github.com/iperov/DeepFaceLab.git /content/DeepFaceLab
!pip install colorama
print("\nDone")

In [ ]:
#@title Install all dependencies and download weights from org repo

upscaling_method = "GPEN" #@param ["GPEN", "GFPGAN", "PSFR-GAN"]

!echo $upscaling_method > /content/upscaling_method

if upscaling_method == "GPEN":
  %cd /content
  !git clone https://github.com/bycloudai/GPEN
  !wget https://github.com/deepartist/DeepFaceLab-Tools/releases/download/v1.0.0/RetinaFace-R50.pth && mv RetinaFace-R50.pth GPEN/weights/
  !wget https://github.com/deepartist/DeepFaceLab-Tools/releases/download/v1.0.0/GPEN-512.pth && mv GPEN-512.pth GPEN/weights/
  #!wget https://github.com/deepartist/DeepFaceLab-Tools/releases/download/v1.0.0/GPEN-1024-Color.pth && mv GPEN-1024-Color.pth GPEN/weights/
  !pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2
  !pip install pip install opencv-python
  %cd GPEN
  !wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
  !sudo unzip ninja-linux.zip -d /usr/local/bin/
  !sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
  !touch "/content/GPEN/__init__.py"
elif upscaling_method == "GFPGAN":
  %cd /content
  # Install basicsr - https://github.com/xinntao/BasicSR
  # We use BasicSR for both training and inference. 
  # Set BASICSR_EXT=True to compile the cuda extensions in the BasicSR - It may take several minutes to compile, please be patient.
  !BASICSR_EXT=True pip install basicsr

  # Install facexlib - https://github.com/xinntao/facexlib
  # We use face detection and face restoration helper in the facexlib package
  !pip install facexlib
  !mkdir -p /usr/local/lib/python3.7/dist-packages/facexlib/weights  # for pre-trained models

  !rm -rf GFPGAN
  !git clone https://github.com/TencentARC/GFPGAN.git
  %cd /content/GFPGAN

  # install extra requirements
  !pip install -r requirements.txt
  !wget https://github.com/deepartist/DeepFaceLab-Tools/releases/download/v1.0.1/GFPGANv1.pth -P experiments/pretrained_models

elif upscaling_method == "PSFR-GAN":
  %cd /content
  !git clone https://github.com/chaofengc/PSFR-GAN.git
  %cd /content/PSFR-GAN
  !wget https://github.com/deepartist/DeepFaceLab-Tools/releases/download/v1.0.2/pretrained_models.zip
  !mkdir -p /content/PSFR-GAN/pretrain_models
  !7z e '/content/PSFR-GAN/pretrained_models.zip' -o'/content/PSFR-GAN/pretrain_models'
  %cd /content
 
  

In [ ]:
#@title create custom upscale function

upscaling_method = !cat /content/upscaling_method
upscaling_method = upscaling_method[0]



if upscaling_method is None or upscaling_method.strip() == "" or "cat" in upscaling_method: 
  raise ValueError("Run cell above, choose and install upscaling_method ")


if upscaling_method == "GPEN":  

  indir = '/content/GPEN/examples/imgs'
  !mkdir -p $indir
  outdir = '/content/GPEN/examples/outs'
  !mkdir -p $outdir
  
  !echo $indir > /content/indir
  
  !echo $outdir > /content/outdir
  
 
  
  def upscale(indir='examples/imgs', outdir= 'examples/outs', model={'name':'GPEN-512', 'size':512}, filetype='jpg'):
    %cd /content/GPEN
    !source /content/GPEN/__init__.py
    from face_enhancement import FaceEnhancement
    import tqdm 
    import glob 
    import cv2
    import numpy as np
    import os 

    

    os.makedirs(outdir, exist_ok=True)

    faceenhancer = FaceEnhancement(size=model['size'], model=model['name'], channel_multiplier=2)
    
    files = sorted(glob.glob(os.path.join(indir, '*.*g'))) #todo other filetype support
    print(f"Found {len(files)} files")
    for file in tqdm.tqdm(files[:]):
        filename = os.path.basename(file)
        
        im = cv2.imread(file, cv2.IMREAD_COLOR) # BGR
        if not isinstance(im, np.ndarray): print(filename, 'error'); continue
        #im = cv2.resize(im, (0,0), fx=1, fy=1) #no rescale

        img, orig_faces, enhanced_faces = faceenhancer.process(im)
        save_filepath = os.path.join(outdir, '.'.join(filename.split('.')[:-1])+f".{filetype}")
        try:
          cv2.imwrite(save_filepath, img)
        except Exception:
          print(f"Could not write file {save_filepath}")
  
  def clear():
    !rm -rf /content/GPEN/examples/imgs
    !mkdir -p /content/GPEN/examples/imgs
    !rm -rf /content/GPEN/examples/outs
    !mkdir -p /content/GPEN/examples/outs
    

elif upscaling_method == "GFPGAN":
  
  indir = '/content/GFPGAN/inputs/uploads'
  !mkdir -p $indir
  outdir = '/content/GFPGAN/results/restored_imgs'
  !mkdir -p $outdir

  !echo $indir > /content/indir
  !echo $outdir > /content/outdir

  def upscale(indir='/content/GFPGAN/inputs/uploads', jpg_output=False):
    %cd /content/GFPGAN/
    import glob
    from pathlib import Path
    import os 
    import cv2

    ext = glob.glob(f"/{indir}/*")[0].split(".")[-1]
    if ext.lower() == "jpg":
      jpg_output=True 
    else:
      jpg_output=False

    !python /content/GFPGAN/inference_gfpgan_full.py --model_path /content/GFPGAN/experiments/pretrained_models/GFPGANv1.pth --test_path $indir --paste_back 
    
    if not jpg_output: return
    files = sorted(glob.glob(f"{outdir}/*png"))
    #print(files)
    for f in files:
      basename = Path(f).stem    
      image = cv2.imread(f)

      # Save .jpg image
      save_path = os.path.join(Path(f).parent, (basename+".jpg"))
      cv2.imwrite(save_path, image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
      !rm $f

  def clear():
    !rm -rf '/content/GFPGAN/results/restored_imgs'
    !mkdir -p '/content/GFPGAN/results/restored_imgs'
    !rm -rf '/content/GFPGAN/inputs/uploads'
    !mkdir -p '/content/GFPGAN/inputs/uploads'
    

elif upscaling_method == "PSFR-GAN":
  
  indir = '/content/inputs/uploads'
  !mkdir -p $indir
  outdir = '/content/test_unalign_results'
  !mkdir -p $outdir

  !echo $indir > /content/indir
  !echo $outdir > /content/outdir

  def upscale(indir='/content/inputs/uploads', outdir='/content/test_unalign_results', jpg_output=False):
    import glob
    from pathlib import Path
    import os 
    import cv2
    %cd /content/PSFR-GAN/
    !python test_enhance_dir_unalign.py --src_dir $indir --results_dir $outdir
    
    # if not jpg_output: return
    files = sorted(glob.glob(f"{outdir}/*/"))
    
    for f in files:
      filename = Path(f).stem
      if os.path.isdir(f):
        !cp $f"/hq_final.jpg" $outdir"/"$filename".jpg"
        !rm -rf $f

  def clear():
    !rm -rf '/content/inputs/uploads'
    !mkdir -p '/content/inputs/uploads'
    !rm -rf '/content/test_unalign_results'
    !mkdir -p '/content/test_unalign_results'


      

# Upscale Process

In [ ]:
#@title 0. Delete old example and result files
clear()
print("\nDone!")

## Upload files
You can upload single or multiple files or your zip aligend folder or faceset.pak

### files and zipped aligned folder

In [ ]:
#@title 1a. Upload one or more images to upscale *OR* 
%cd $indir
from google.colab import files
uploaded = files.upload()
print("\nDone!")

In [ ]:
#@title 1b. Upload aligned.zip and extract to imgs *OR*
%cd /content/
!rm aligned.zip
from google.colab import files
uploaded = files.upload()
%cd $indir
!7z e '/content/aligned.zip' -o $indir
print("\nDone!")

In [ ]:
#@title 1c. copy aligned.zip from your drive and extract to imgs
import os
file_on_drive = "/content/drive/MyDrive/aligned.zip" #@param {type:"string"}
%cd /content/
!rm aligned.zip
!rsync --info=progress2 $file_on_drive /content/
!7z e '/content/aligned.zip' -o $indir
print("\nDone!")

### faceset.pak

In [ ]:
#@title 2b. Upload faceset.pak and extract to imgs *OR*
%cd $indir
!rm faceset.pak
from google.colab import files
uploaded = files.upload()
%cd /content
cmd = f"DeepFaceLab/main.py util --input-dir {indir} --unpack-faceset"
!python $cmd
print("Unpacking done!")
print("\nDone!")

In [ ]:
#@title 2d. copy faceset.pak from your drive and extract to imgs
import os

file_on_drive = "/content/drive/MyDrive/faceset.pak" #@param {type:"string"}
%cd /content/
!rm faceset.pak
!cp $file_on_drive $indir
cmd = f"DeepFaceLab/main.py util --input-dir {indir} --unpack-faceset"
!python $cmd
print(f"Unpacking done!")
print("\nDone!")

## Upscaling


In [ ]:
#@title 3. Optional:Use DFL resize if original input faceset if too small
%cd /content/
cmd = f"DeepFaceLab/main.py facesettool resize --input-dir {indir}"
!python $cmd

In [ ]:
#@title 4. Save DFL metadata and Start upscaling and restore

#@markdown Ignore errors if you dont use DFLIMG
%cd /content/
cmd = f"DeepFaceLab/main.py util --input-dir {indir} --save-faceset-metadata"
!python $cmd
!mv $indir"/meta.dat" $outdir
print("Start upscaling")
upscale()
print("Upscaling done")
%cd /content/
cmd = f"DeepFaceLab/main.py util --input-dir {outdir} --restore-faceset-metadata"
!python $cmd
print("\nDone!")

In [ ]:
#@title 4. Optional:Use DFL resize on output dir
%cd /content/
cmd = f"DeepFaceLab/main.py facesettool resize --input-dir {outdir}"
!python $cmd

## Save your enhanced files

### Zips

In [ ]:
#@title 5a. Zip enhanced faces
%rm /content/dfl_enhanced.zip
%rm $outdir"/faceset.pak"
!7z a -tzip /content/dfl_enhanced.zip -w $outdir"/*"
print("\nDone!")

In [ ]:
#@title 5b. Download dfl_enhanced.zip *OR*
from google.colab import files
files.download("/content/dfl_enhanced.zip")
print("\nDone!")

In [ ]:
#@title 5c. Copy dfl_enhanced.zip to gdrive
from google.colab import files
folder_on_drive = "/content/drive/MyDrive/" #@param {type:"string"}
!rsync -ah --info=progress2 /content/dfl_enhanced.zip  $folder_on_drive
print("\nDone!")

### Faceset.pak

In [ ]:
#@title 6a. Repack to faceset.pak 
%cd /content
import glob 
files = glob.glob(f"{outdir}/*jpg")
if len(files) > 0:
  %rm $outdir"/faceset.pak"
  cmd = f"DeepFaceLab/main.py util --input-dir {outdir} --pack-faceset"
  !python $cmd
  !cp $outdir"/faceset.pak" /content/faceset.pak
else: 
  print('No imgs found')


In [ ]:
#@title 6b. Download faceset.pak *OR*
from google.colab import files
files.download(f"{outdir}/faceset.pak")
print("\nDone!")

In [ ]:
#@title 6c. Copy faceset.pak to gdrive
from google.colab import files
folder_on_drive = "/content/drive/MyDrive/" #@param {type:"string"}
!rsync -ah --info=progress2 $outdir"/faceset.pak" $folder_on_drive
print("\nDone!")